# Introduction

This notebook aligns individual exposures to Gaia, then drizzles them together. It follows from the notebook titled *1) Drizzle Images*, which creates a drizzled image that is not aligned to Gaia, but is used to determine offsets from Gaia.

AstroDrizzle performs distortion corrections. Therefore, the individual flat-fielded exposures (FLC files) must be aligned to Gaia before drizzling. This notebook is organized as follows:
<ol>
  <li>Import Packages</li>
  <li>Copy individual exposures to working directory</li>
  <li>Run SEP (SourceExtractor for Python) on drizzled image</li>
  <li>Query Gaia sources close to field of view</li>
  <li>Calculate offsets between positions of stars in drizzled image to Gaia positions</li>
  <li>Update the world coordinate system information in the headers of the FLC files</li>
  <li>Drizzle aligned-FLC files to attain final aligned image</li>
</ol>

# Import Packages

In [ ]:
import pandas as pd
import os
from shutil import copyfile
from glob import glob
from astropy.io import fits
import astropy.units as u
from astropy.time import Time
from drizzlepac import astrodrizzle

# Import Gaia-alignment pipeline packages
from get_gaia_and_sep_sources import get_sources # Create source catalog (using SEP) and Gaia catalog from input image
from get_offsets import get_offsets # Calculate offset between source catalog and reference (i.e., Gaia) catalog
from update_wcs_flc import calc_new_wcs, update_wcs # Update the WCS information in the individual exposures

# Define parent directory
# The working directory is changed later in the Notebook
parent = os.getcwd()

# Copy individual exposures to working directory

First copy all individual exposures that need to be aligned into a new working directory. This ensures that we can modify them as needed without affecting the original versions.

In [ ]:
# Define the list of files that were previously aligbed to one another, but not aligned to Gaia
# These are created in the notebook titled 1) Drizzle Images
file_list = glob('drizzled_products_unaligned/*_flc.fits')
file_list

In [ ]:
# Ensure you are in parent directory
os.chdir(parent)

# Define a directory where the aligment to Gaia will take place
alinging_wdir = 'aligned_flcs/'

# Create the directory if it doesn't already exist
if os.path.exists(alinging_wdir) == False:
    os.mkdir(alinging_wdir)

# Copy files to this directory
for og_file in file_list:
    
    new_path = os.path.join(alinging_wdir, os.path.basename(og_file).replace('_flc', '_flc_aligned'))
    
    print('\n')
    print('Original File:', og_file)
    print('New Path:', new_path)
    
    copyfile(og_file, new_path)

In [ ]:
# Define list of files that will be aligned
to_align_files = glob('aligned_flcs/*_flc_aligned*')
to_align_files

# Run SEP (SourceExtractor for Python) on drizzled image

Detect sources using [SourceExtractor for Python (SEP)](https://sep.readthedocs.io/en/stable/) to detect sources in the unaligned drizzled image. This image was created in the notebook titled $\textit{1) Drizzle Images}$.

This is done using a class named "get_sources" that was created for this pipeline. This Python class takes the file name of the drizzled image as input, then runs SEP using the "run_sep" function. Please refer to the source code for a full description of available parameters.

In [ ]:
# Define name of unaligned drizzled image
drc_file = 'drizzled_products_unaligned/jdkq01_drc_sci.fits'

# Get sources
source_class = get_sources(drc_file)
object_df = source_class.run_sep(save_sep = 'sep_sources_unaligned.csv', 
                                 thresh = 5, deblend_cont = 0.06, 
                                 flux_limits = [0, 10000])

# Query Gaia sources

Query Gaia sources using the same "get_sources" class mentioned above.

In [ ]:
# Get time of observation (to account for proper motion of Gaia sources)
expstart = fits.open(drc_file)[0].header['EXPSTART']
drc_time = Time(expstart, format = 'mjd').to_value('jyear')

# Query sources
gaia_query = source_class.run_gaia_query(drc_time = drc_time, save_gaia = 'gaia_sources.csv', 
                    gaia_flux_limits = [0,5000], match_drc = True, 
                    object_df = object_df,
                    new_ra_col = 'ra', new_dec_col = 'dec')

# Align images

In [ ]:
# Read in Gaia dataframe
gaia_df = pd.read_csv('gaia_sources.csv')

Calculate the x, y, and theta offsets between the true Gaia positions and the unaligned positions in the drizzled image. Then, update the world coordinate system information in the header of each exposure to align them to Gaia.

This is performed using the "get_offsets", "calc_new_wcs", and "update_wcs" functions that were created for this pipeline. They produce plots that show the uncorrected and corrected positions of Gaia sources in the image.

In [ ]:
# Loop through every exposure
for unaligned_file in to_align_files:

    # Loop through both science extensions (located at the first and fourth image extensions)
    for ext in [1,4]:
        
        print('-----------------------------------------------------------------')
        print(unaligned_file, ext)
        print('-----------------------------------------------------------------')

        # Calculate the x, y and theta offsets
        outputs, drc_matches, gaia_matches = get_offsets(file = unaligned_file, 
                                                         sciext = ext, 
                                                         refcat_df = gaia_df, 
                                                         image_src_df = object_df, 
                                                         pixel_scale = 0.05, 
                                                         num_it = 3, 
                                                         origin = 0).main()
        
        # Calculate the new central pixel RA, Dec, position angle, and CD matrix based on
        # the x, y and theta offsets above
        new_refra, new_refdec, new_pa_deg, new_cd1_1, new_cd1_2, new_cd2_1, new_cd2_2 = calc_new_wcs(unaligned_file, sciext = ext, 
                                                                                                     dx = outputs['dx'], dy = outputs['dy'], 
                                                                                                     theta = outputs['theta'])
        
        # Apply transformations to science ext, DQ ext, and ERR ext
        for ext_i in [ext, ext+1, ext+2]:
            print(ext_i)
            update_wcs(unaligned_file, ext_i, 
                       new_refra, new_refdec, new_pa_deg.value, 
                       new_cd1_1, new_cd1_2, new_cd2_1, new_cd2_2)

# Drizzle

Create a new drizzled image using the aligned exposures.

In [ ]:
os.chdir(parent)

# Create a new directory
driz_dir = 'drizzled_products_aligned'

if os.path.exists(driz_dir) == False:
    os.mkdir(driz_dir)
    
print('!!! Changing directory to {}'.format(driz_dir))
os.chdir(driz_dir)

In [ ]:
# Make list of aligned exposures
corrected_flcs = glob('../aligned_flcs/*_flc_aligned.fits')
corrected_flcs

In [ ]:
# Copy all aligned exposures to working directory
for file in corrected_flcs:
    
    print(file, os.path.basename(file))
    
    copyfile(file, os.path.basename(file))

In [ ]:
# Drizzle

input_flcs = glob('*_aligned.fits')
input_flcs.sort()

astrodrizzle.AstroDrizzle(input_flcs,
                          output='jdkq01',
                          final_bits = [1,2,32,64],
                          final_kernel = 'square',
                          final_scale = 0.06,
                          build = False, 
                          combine_type = 'median', 
                          preserve = False,
                          combine_nhigh = 1, 
                          clean = True, 
                          driz_cr_corr = True, 
                          driz_cr_snr = '3.5 3.0',
                          final_wht_type = 'WHT', 
                          runfile='driz_log.txt')

# Refind sources

Calculate the offsets one last time to ensure alignment was performed properly.

In [ ]:
os.chdir(parent)

In [ ]:
# Detect sources using SEP
aligned_drc_file = 'drizzled_products_aligned/jdkq01_drz_sci.fits'
aligned_source_class = get_sources(aligned_drc_file)
aligned_object_df = aligned_source_class.run_sep(save_sep = 'sep_sources_aligned.csv', 
                                 thresh = 5, deblend_cont = 0.06, 
                                 flux_limits = [0, 10000])

In [ ]:
# Calculate offsets
outputs, drc_matches, gaia_matches = get_offsets(file = aligned_drc_file, 
                                                 sciext = 0, 
                                                 refcat_df = gaia_df, 
                                                 image_src_df = aligned_object_df, 
                                                 pixel_scale = 0.06, num_it = 3, 
                                                 distort_corr = True, 
                                                 origin = 0).main()

In [ ]:
# Plot sources used for alignment
source_class = get_sources(aligned_drc_file)
source_class.plot_sources(drc_matches[0], drc_matches[1], 
                          gaia_matches['x'], gaia_matches['y'], show_gaia = True)